## 라이브러리 설치

In [2]:
!pip3 install gradio

# gradio 구현(1) : Hello!

In [25]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet,
                    allow_flagging='manual',
                    inputs = "text",
                    outputs = "text")

demo.launch(server_name = "0.0.0.0",
            server_port = 700,
            auth = ("noname", "passwdl"),
            share=True)

Running on local URL:  http://0.0.0.0:700
Running on public URL: https://42345c90e8bf720163.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


- launch()에서 share=True로 지정하면 gradio에서 제공하는 소스를 사용하게 된다.

# gradio 구현(2) : 분류 모델

In [4]:
import tensorflow as tf

inception_net = tf.keras.applications.MobileNetV2()

14548992/14536120 [==============================] - 0s 0us/step


In [14]:
import requests

response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def classify_image(inp):
    inp = inp.reshape((-1, 224, 224, 3))
    inp = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
    prediction = inception_net.predict(inp).flatten()
    confidences = {labels[i]: float(prediction[i]) for i in range(1000)}
    return confidences    

In [33]:
import gradio as gr

classification = gr.Interface(fn=classify_image,
                              inputs = gr.Image(shape=(224,224)),
                              outputs = gr.Label(num_top_classes=3))

classification.launch(server_name = "0.0.0.0",
                      server_port = 600,
                      auth = ("noname", "passwdl"),
                      share=True)

Running on local URL:  http://0.0.0.0:600
Running on public URL: https://d7fcaa8cb3ba1bafc9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/gradio/routes.py", line 439, in run_predict
    output = await app.get_blocks().process_api(
  File "/opt/conda/lib/python3.9/site-packages/gradio/blocks.py", line 1384, in process_api
    result = await self.call_function(
  File "/opt/conda/lib/python3.9/site-packages/gradio/blocks.py", line 1089, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/opt/conda/lib/python3.9/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "/opt/conda/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "/opt/conda/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "/opt/conda/lib/python3.9/site-packages/gradio/utils.py", line 700, in wrapper
    resp

# 퍼스널 컬러 앱 구현

In [ ]:
# knn / zero shot의 경우

## 1. 데이터 수집(라벨링 된)
## 2. 데이터 입력
## 3. 모델이 웜/쿨인지 결과 출력

In [32]:
# cifar10으로 예상 모형 만들기

import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def classify_image(image):
    # Preprocess the image
    image = image.reshape((1, 32 * 32 * 3)) / 255.0

    # Load the trained model
    model = keras.models.load_model('keras_cifar10_model.h5')

    # Perform prediction
    prediction = model.predict(image)
    class_index = np.argmax(prediction)
    class_name = class_names[class_index]

    return class_name

# Load CIFAR-10 dataset and split into train, validation, and test sets
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train_full,
                                                  y_train_full,
                                                  test_size=0.25,
                                                  random_state=42)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], 32 * 32 * 3)
x_val = x_val.reshape(x_val.shape[0], 32 * 32 * 3)
x_test = x_test.reshape(x_test.shape[0], 32 * 32 * 3)

x_train = x_train / 255.
x_val = x_val / 255.
x_test = x_test / 255.

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Define the model architecture
model = keras.models.Sequential()
model.add(layers.Input(shape=(3072,)))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax', name='output'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
check_point_cb = callbacks.ModelCheckpoint('keras_cifar10_model.h5', save_best_only=True)
early_stopping_cb = callbacks.EarlyStopping(patience=3,
                                            monitor='val_loss',
                                            restore_best_weights=True)

# Train the model
history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[check_point_cb, early_stopping_cb])

# Define the Gradio interface
iface = gr.Interface(fn=classify_image,
                     inputs=gr.inputs.Image(shape=(32, 32)),
                     outputs=gr.outputs.Label(num_top_classes=2))

# Launch the interface
iface.launch(share=True,
             server_name = "0.0.0.0",
             server_port = 300,
             auth = ("noname", "passwdl"))

Epoch 1/40
74/74 [==============================] - 3s 24ms/step - loss: 2.4594 - accuracy: 0.2381 - val_loss: 8.2493 - val_accuracy: 0.1582
Epoch 2/40
74/74 [==============================] - 1s 16ms/step - loss: 2.0836 - accuracy: 0.3036 - val_loss: 3.5494 - val_accuracy: 0.1947
Epoch 3/40
74/74 [==============================] - 1s 16ms/step - loss: 1.8880 - accuracy: 0.3479 - val_loss: 2.6983 - val_accuracy: 0.2472
Epoch 4/40
74/74 [==============================] - 1s 16ms/step - loss: 1.7694 - accuracy: 0.3812 - val_loss: 1.8561 - val_accuracy: 0.3471
Epoch 5/40
74/74 [==============================] - 1s 16ms/step - loss: 1.6973 - accuracy: 0.4047 - val_loss: 1.7415 - val_accuracy: 0.3750
Epoch 6/40
74/74 [==============================] - 1s 16ms/step - loss: 1.6479 - accuracy: 0.4222 - val_loss: 1.7117 - val_accuracy: 0.3881
Epoch 7/40
74/74 [==============================] - 1s 15ms/step - loss: 1.6072 - accuracy: 0.4298 - val_loss: 1.7970 - val_accuracy: 0.3489
Epoch 8/40
74

/tmp/ipykernel_31/564398416.py:85: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(shape=(32, 32)),
/tmp/ipykernel_31/564398416.py:85: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(shape=(32, 32)),
/tmp/ipykernel_31/564398416.py:86: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Label(num_top_classes=3))
/tmp/ipykernel_31/564398416.py:86: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  outputs=gr.outputs.Label(num_top_classes=3))


Running on local URL:  http://0.0.0.0:300
Running on public URL: https://29dc69126715bd74ee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


- 위에서는 cifar10의 딥러닝 학습 분류 모델을 이용한 것을 gradio로 구현한 코드이다.
    퍼스널 컬러 분류 코드가 만들어지면 위와 같이 구현할 수 있을 것이다.

- 퍼스널 컬러의(구현한다면) 퍼센테이지(%)를 확인할 수 있게 하기 위해 label의 top2를 나타내게 하고 싶은데 데모에선 됐는데 위에선 표시가 안 됐다. 다시 코드를 살펴보고 수정해야 할 것 같음

## 퍼컬 앱 구현

In [ ]:
# 퍼스널 컬러 분류 함수 만들기

def personal_color(img):
    img = img.reshape((-1, 244, 244, 3))
#     model = keras.models.load_model('unet_model.h5')
#     segmentation = model.predict(img)

In [ ]:
# gradio로 시각화? 웹 구현
import gradio as gr

classification = gr.Interface(fn=personal_color,
                              inputs="image",
                              outputs="label",
                              examples = [])

classification.launch(share=True,
                      server_name = "0.0.0.0",
                      server_port = 8088,
                      auth = ("noname", "passwdl"))

- 위는 아직 퍼스널 컬러 코드가 완성되지 않아 비워두었다.

# Reference

- 분류 모델 공식 홈페이지 데모 버전 (https://www.gradio.app/guides/image-classification-in-tensorflow)